In [147]:
import numpy as np
from sklearn.datasets import load_digits
import pylab as plt
from collections import OrderedDict
from numpy.linalg import lstsq, norm
from sklearn import cross_validation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
%matplotlib inline

# 3) automatic feature selection for regression

## 3.1) orthogonal matching pursuit

In [192]:
def omp_regression(X, y, t):
    """
    takes an NxD feature matrix X and an an Nx1 outcome y. 
    selects t features that maximally reduce the residuals.
    X needs to be standardized.
    """
    
    # dimensions
    N = X.shape[0]
    D = X.shape[1]
    
    #initialization
    A = [] #set of active columns in the order as selected
    B = [i for i in range(D)] # set of inactive columns
    r = np.array(y) #residual
    current_X = np.empty((N, 1))
    current_beta = None
    
    #iteratively find feature that is maximally correlated to the current residual
    for i in range(t):
        
        #find the feature. j is the index in B!
        col_reductions = [np.dot(X[:,k], y) for k in B]
        j = np.argmax(np.absolute(col_reductions))
        
        #move it from A to B
        A.append(B[j])
        del B[j]
        
        #updating current_X
        col = X[:, j]
        col = np.expand_dims(col, 0)
        if i == 0:
            current_X = col.T
        else:
            current_X = np.concatenate((current_X, col.T), 1)
            
        #solving current least squares problem
        lstsq_results = lstsq(current_X, y)
        current_beta = lstsq_results[0]
        
        #updating residuals
        r = y - np.dot(current_X, current_beta)
    
    #generating beta to return with t non-zero entries
    beta = np.zeros(D)
    for i in range(t):
        dim = A[i]
        beta[dim] = current_beta[i]
        
    return beta, A

## 3.2) LDA as sparse regression on digits 1 and 7

In [ ]:
def prepare_two_classes(raw_features, raw_labels, class1, class2, test_proportion):
    '''function to prepare two classes for sparse regression. returning standardized data'''
    
    #standardize data
    std_features = scale(raw_features)
    
    #splitting
    x_train, x_test, y_train, y_test = train_test_split(std_features, \
                                        raw_labels, test_size=test_proportion)
    
    #class extraction
    x_train_class1 = x_train[np.where(y_train == class1)]
    x_train_class2 = x_train[np.where(y_train == class2)]
    x_test = x_test[np.where((y_test==class1)|(y_test==class2))]
    y_test = y_test[np.where((y_test==class1)|(y_test==class2))]
    
    return x_train_class1, x_train_class2, x_test, y_test

In [156]:
#loading data set
digits = load_digits()

data = digits['data']
images = digits['images']
target = digits['target']
target_names = digits['target_names']

#extracting ones and sevens and dividing into training and test set
x_1, x_7, x_test, y_test = prepare_two_classes(data, target, 1, 7, 0.42)

#concatenating training data to a single data matrix
x_train = np.concatenate((x_1, x_7), 0)

#generating corresponding output values, 1 for digit one and -1 for digit seven
y_train = np.concatenate((np.full(len(x_1), 1), np.full(len(x_7), -1)))

/Users/lucasmoeller/anaconda/lib/python3.4/site-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full(107, 1) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)
/Users/lucasmoeller/anaconda/lib/python3.4/site-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full(104, -1) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)


In [185]:
def two_class_error(guess, truth, class1=1, class2=7):
    '''returns the total error rate and confusion matrix'''
    
    #absolute true and false classifications
    false_class1 = len(guess[np.where((truth != class1) & (guess == class1))])
    true_class1 = len(guess[np.where((truth == class1) & (guess == class1))])
    total_class1 = len(truth[np.where(truth == class1)])
    false_class2 = len(guess[np.where((truth != class2) & (guess == class2))])
    true_class2 = len(guess[np.where((truth == class2) & (guess == class2))])
    total_class2 = len(truth[np.where(truth == class2)])
    false_total = len(guess[np.where(truth != guess)])
    
    #relative true and false classifications
    error_rate = false_total/len(guess)
    confusion_mat = np.array([[true_class1/total_class1, false_class2/total_class2], \
                                [false_class1/total_class1, true_class2/total_class2]])
    
    return error_rate, confusion_mat

In [193]:
def test_one_seven(x_train, y_train, x_test, y_test, feat_numbers):
    '''evaluates the classification for different feature numbers'''
    
    #evaluation for every t
    for t in feat_numbers:
        
        #regression / training
        beta, features = omp_regression(x_train, y_train, t)
        
        #classifying test set
        labels = np.dot(x_test, beta)
        #translating results
        estimates = np.full(len(labels), 1)
        for i in range(len(labels)):
            if labels[i] < 0:
                estimates[i] = 7
                
        #evaluation
        err, conf = two_class_error(estimates, y_test)
        
        #printing results
        print('\nnumber of features:', t)
        print('selected features:', features)
        print('total error rate:', err)
        print('confusion matrix:\n', conf)
        
    return

test_one_seven(x_train, y_train, x_test, y_test, [20, 18, 15, 12, 10, 5, 3, 2])        


number of features: 20
selected features: [60, 19, 52, 61, 30, 53, 27, 6, 10, 14, 38, 20, 7, 3, 58, 29, 37, 26, 5, 63]
total error rate: 0.23333333333333334
confusion matrix:
 [[ 0.62666667  0.37333333]
 [ 0.09333333  0.90666667]]

number of features: 18
selected features: [60, 19, 52, 61, 30, 53, 27, 6, 10, 14, 38, 20, 7, 3, 58, 29, 37, 26]
total error rate: 0.24
confusion matrix:
 [[ 0.57333333  0.42666667]
 [ 0.05333333  0.94666667]]

number of features: 15
selected features: [60, 19, 52, 61, 30, 53, 27, 6, 10, 14, 38, 20, 7, 3, 58]
total error rate: 0.013333333333333334
confusion matrix:
 [[ 0.98666667  0.01333333]
 [ 0.01333333  0.98666667]]

number of features: 12
selected features: [60, 19, 52, 61, 30, 53, 27, 6, 10, 14, 38, 20]
total error rate: 0.02666666666666667
confusion matrix:
 [[ 0.97333333  0.02666667]
 [ 0.02666667  0.97333333]]

number of features: 10
selected features: [60, 19, 52, 61, 30, 53, 27, 6, 10, 14]
total error rate: 0.03333333333333333
confusion matrix:
 [

/Users/lucasmoeller/anaconda/lib/python3.4/site-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full(150, 1) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)


It is necessary to standardize the data. Otherwise the projection of features onto the residual vector in OMP is totally out of bounds.
In exercise 2 we manually chose features 60 and 10 (by looking at mean and standard deviation of both classes).
OMP chooses 60 and 19 which makes sense when looking back at the error bar plot we made in exercise two. These two features are the two whose distributions are the furthest appart from another. 